<a href="https://colab.research.google.com/github/Tena-rin/Kintsugi-Splitter/blob/main/Scripts/step4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# step4: Fragment segmentation

## これは何？
このノートは、破片分割を行うための実装です。

## できること
- 1. step3で出力された点群データ(o_color.ply)を入力として読み込む
- 2. 色情報をもとに金継ぎ線の削除を行う
- 3. 金継ぎ線の太さを測り、パラメータを自動生成
- 4. DBSCANを用いて破片分割を行う
- 5. pieces_kintsugiファイルの中に破片ごとにply形式で保存

## 実行方法
上から順番にセルを実行してください。
ただし、金継ぎ線の削除の際に用いる色の範囲に関しては、step2で採用された色をもとに行ってください。

In [ ]:
!pip install open3d
! pip show open3d
import open3d as o3d
import os
import math
import numpy as np
import cv2
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors

In [ ]:
# ========= 入力と出力 =========
INPUT_FILE = "o_color.ply"
OUTPUT_DIR = "pieces_kintsugi"
os.makedirs(OUTPUT_DIR, exist_ok=True)

pcd = o3d.io.read_point_cloud(INPUT_FILE)
points = np.asarray(pcd.points)
colors = np.asarray(pcd.colors)

print("points:", points.shape, "colors:", colors.shape)

bgr = (colors[:, ::-1] * 255).astype(np.uint8)
hsv = cv2.cvtColor(bgr.reshape(-1,1,3), cv2.COLOR_BGR2HSV).reshape(-1,3)

lower = np.array([35, 40, 40])
upper = np.array([85, 255, 255])

mask_green = ((hsv[:,0] >= lower[0]) & (hsv[:,0] <= upper[0]) &
              (hsv[:,1] >= lower[1]) & (hsv[:,2] >= lower[2]))

print("green count:", mask_green.sum())

points_nogreen = points[~mask_green]
colors_nogreen = colors[~mask_green]

points_green = points[mask_green]
colors_green = colors[mask_green]

pcd_nogreen = o3d.geometry.PointCloud()
pcd_nogreen.points = o3d.utility.Vector3dVector(points_nogreen)
pcd_nogreen.colors = o3d.utility.Vector3dVector(colors_nogreen)
o3d.io.write_point_cloud(os.path.join(OUTPUT_DIR, "without_green.ply"), pcd_nogreen)

pcd_green = o3d.geometry.PointCloud()
pcd_green.points = o3d.utility.Vector3dVector(points_green)
pcd_green.colors = o3d.utility.Vector3dVector(colors_green)
o3d.io.write_point_cloud(os.path.join(OUTPUT_DIR, "only_green_removed.ply"), pcd_green)

print(f"{OUTPUT_DIR}/without_green.ply（緑を除去した点群）")
print(f"{OUTPUT_DIR}/only_green_removed.ply（除去された緑の点群）")

INPUT_FILE = "pieces_kintsugi/only_green_removed.ply"

pcd = o3d.io.read_point_cloud(INPUT_FILE)
points = np.asarray(pcd.points)

print("点数:", len(points))

k = 10
nbrs = NearestNeighbors(n_neighbors=k+1).fit(points)
distances, indices = nbrs.kneighbors(points)

mean_local_dist = distances[:, 1:].mean(axis=1)

local_diameters = mean_local_dist * 2

min_thickness = local_diameters.min()
max_thickness = local_diameters.max()
avg_thickness = local_diameters.mean()
std_thickness = local_diameters.std()

print("最も細い部分の推定直径:", min_thickness)
print("平均的な太さ:", avg_thickness)
print("最も太い部分の推定直径:", max_thickness)
print("太さのばらつき (std):", std_thickness)

m = math.ceil(min_thickness * 1000) / 1000
print(m)

from sklearn.cluster import DBSCAN

clustering = DBSCAN(eps=0.016, min_samples=20).fit(points_nogreen)
labels = clustering.labels_

cluster_id = 0

for lbl in set(labels):
    if lbl == -1:
        continue

    mask = (labels == lbl)
    cluster_points = points_nogreen[mask]
    cluster_colors = colors_nogreen[mask]

    print(points.sum())
    if len(cluster_points) > (len(points_nogreen)/2):
        print(f"クラスタ {lbl} は {len(cluster_points)} 点 → 再クラスタリング開始")

        sub_clustering = DBSCAN(eps=0.01, min_samples=20).fit(cluster_points)
        sub_labels = sub_clustering.labels_

        for sub_lbl in set(sub_labels):
            if sub_lbl == -1:
                continue
            sub_mask = (sub_labels == sub_lbl)
            sub_points = cluster_points[sub_mask]
            sub_colors = cluster_colors[sub_mask]

            if len(sub_points) < 30:
                continue

            piece = o3d.geometry.PointCloud()
            piece.points = o3d.utility.Vector3dVector(sub_points)
            piece.colors = o3d.utility.Vector3dVector(sub_colors)

            filename = os.path.join(OUTPUT_DIR, f"piece_{cluster_id}.ply")
            o3d.io.write_point_cloud(filename, piece)
            print(f"保存: {filename} ({len(sub_points)} 点)")

            cluster_id += 1

    else:
        if len(cluster_points) < 0:
            continue

        piece = o3d.geometry.PointCloud()
        piece.points = o3d.utility.Vector3dVector(cluster_points)
        piece.colors = o3d.utility.Vector3dVector(cluster_colors)

        filename = os.path.join(OUTPUT_DIR, f"piece_{cluster_id}.ply")
        o3d.io.write_point_cloud(filename, piece)
        print(f"保存: {filename} ({len(cluster_points)} 点)")

        cluster_id += 1